In [92]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
# コールバックに必要なライブラリのインポート
from dash.dependencies import Input, Output, State

データの読み込み

In [93]:
df_pref = pd.read_csv('../data/cleaning_pref.csv')
display(df_pref.head(3))

df_kind = pd.read_csv('../data/kinds_of_rank.csv').reset_index().rename(columns={'index':'rank'})
display(df_kind.head(3))

,name,point,address,latitude,longitude,parking,holiday,mon_holiday,tue_holiday,wed_holiday,thu_holiday,fri_holiday,sat_holiday,sun_holiday
0,麺屋 彩未,96.632,〒062-0010 北海道札幌市豊平区美園十条5丁目3-12,43.035251,141.381909,駐車場あり,月曜、火曜水曜の夜、月2回不定休有,1,1,1,0,0,0,0
1,ラーメン二郎 札幌店,95.182,〒060-0806 北海道札幌市北区北六条西8-8-11,43.067395,141.343821,駐車場なし,日曜、祝日不定休,0,0,0,0,0,0,1
2,すみれ 札幌中の島本店,94.228,〒062-0922 北海道札幌市豊平区中の島二条4-7-28,43.032555,141.361599,駐車場あり,不定休,0,0,0,0,0,0,0


,rank,ラーメン,つけ麺,汁なし,冷やし中華,醤油,味噌,塩,豚骨,豚骨醤油,豚骨魚介,鶏白湯,煮干し,担々麺,カレー,塩豚骨
0,0,麺屋吉左右,中華蕎麦 とみ田,ラーメン二郎 横浜関内店,千里眼,らぁ麺 飯田商店 湯河原本店,三ん寅,SOBAHOUSE 金色不如帰 新宿御苑本店,中華蕎麦 ひら井,ラーメン二郎 ひばりヶ丘駅前店,中華蕎麦 とみ田,麺屋 一燈,煮干中華ソバ イチカワ,創作麺工房 鳴龍,狼煙 〜NOROSHI〜,田中そば店 秋葉原店
1,1,らぁ麺 飯田商店 湯河原本店,中華蕎麦うゑず,狼煙 〜NOROSHI〜,麺恋処 いそじ,中華そば 髙野,大島,宍道湖しじみ中華蕎麦 琥珀 東京本店,中華蕎麦 とみ田,厚木家,麺屋吉左右,銀座 篝 本店,中華そば 西川,麺庵ちとせ,自家製麺 福は内,麺屋 蓮花
2,2,ラーメン二郎 ひばりヶ丘駅前店,Tokyo Style Noodle ほたて日和,燦燦斗,饗 くろ㐂,人類みな麺類,味噌麺処 花道庵,Tokyo Style Noodle ほたて日和,博多長浜らーめん 田中商店,ラーメン二郎 三田本店,中華蕎麦うゑず,麺堂 稲葉,中華ソバ 伊吹,ほっこり中華そば もつけ,ラーメン燈郎,大ちゃんらーめん


スタイルシートの作成.  
checkbox要素の準備

In [94]:
# bootstrapスタイルシートの読み込み
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# optionに設定される要素の準備
vars_parking = [park for park in np.unique(df_pref['parking'].values)]
print(vars_parking)
week_dict = {
  '月':'mon_holiday',
  '火':'tue_holiday',
  '水':'wed_holiday',
  '木':'thu_holiday',
  '金':'fri_holiday',
  '土':'sat_holiday',
  '日':'sun_holiday'
}
print(week_dict)

['情報なし', '駐車場あり', '駐車場なし']
{'月': 'mon_holiday', '火': 'tue_holiday', '水': 'wed_holiday', '木': 'thu_holiday', '金': 'fri_holiday', '土': 'sat_holiday', '日': 'sun_holiday'}


サイドバーのコーディング

In [95]:
# サイドバーのレイアウト定義
sidebar = html.Div(
  [
    # タイトルの表示
    dbc.Row(
      [
        html.H5('RamenDB Plotly', style={'margin-top': '12px', 'margin-left': '24px'})
      ],
      style={"height": "5vh"},
      className='bg-primary text-white font-italic'
    ),
    dbc.Row(
      [
        # 駐車場の有無,空いている店舗を選択できる
        html.Div(
          [
            html.P('駐車場の有無', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Checklist(
              id='check_parking',
              # 表示する要素辞書形式で指定
              options=[{'label': x, 'value': x} for x in vars_parking], 
              # 初期で指定する要素の指定
              value=['駐車場あり', '駐車場なし', '情報なし'],
              # 要素の縦表示か横表示かの指定
              inline=False,
              style={'width': '320px'}
            ),
            html.P('定休日の選択', style={'margin-top': '16px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Checklist(
              id='check_holiday',
              options=[{'label': key, 'value': value} for key, value in week_dict.items()], 
              value=[], 
              inline=True,
              style={'width': '50px'}
            ),
            html.Button(id='my-button', n_clicks=0, children='apply', style={'margin-top': '16px'}, className='bg-dark text-white'),
            html.Hr()
          ]
        )
      ],
      style={'height': '50vh', 'margin': '8px'}
    )
  ]
)

メイン部分のマップの描画

In [98]:
# hover時に表示するレコードを指定
cols = df_pref.columns.drop(['address', 
                             'latitude', 
                             'longitude', 
                             'mon_holiday',
                             'tue_holiday',
                             'wed_holiday',
                             'thu_holiday',
                             'fri_holiday',
                             'sat_holiday',
                             'sun_holiday'])

# scatter_mapboxグラフの作成
fig = px.scatter_mapbox(
  # データフレームと、緯度、軽度の指定
  df_pref, 
  lat="latitude", 
  lon="longitude", 
  # ホバー時の表示設定
  hover_name="name",
  hover_data=cols,
  color_discrete_sequence=["#636EFA"], 
  opacity=0.3,
  #詳細設定
  zoom=3, 
  height=450,
  width=450
)
# mapの種類を指定
fig.update_layout(mapbox_style="carto-positron")
# 余白の調整
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

コンテンツ部分の画像の貼り付け

In [99]:
content = html.Div(
  [
    dbc.Row(
      [
        html.P('Map'),
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        dcc.Graph(id='ramen_map', figure=fig)
      ]
    )
  ]
)

アプリの起動

In [97]:
# 画面全体のgridの縦比を、1：3に分割する
app.layout = dbc.Container(
  [
    dbc.Row(
      [
        dbc.Col(sidebar, width=3, className='bg-light'),
        dbc.Col(content, width=9)
      ],
      style={"height": "100vh"}
    ),
  ],
  # 不要な余白の除去
  fluid=True
)

if __name__ == "__main__":
  app.run_server(debug=True, port=1234)
